In [1]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze including the benchmark
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM', 'PLTR', 'QUBT', 'IONQ', 'APLD', 'LMND', 'VRT', 'RKLB', 'AFRM', 'FLR', 'ESOA', 'BKNG', 'HQY', 'NU', 'VNO', 'MP', 'SMR', 'IDCC', 'RVMD', 'DHI', 'PANW', 'DASH', 'CHWY', 'WPM',
           'CL', 'LMT', 'TKO', 'T', 'SE', 'GDX', 'COST', 'GOOG', 'SLV', 'NEE', 'CCJ', 'IAU', 'LNTH', 'GEVO']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # Adjust this as needed
rolling_window = 20   # Define the window size for rolling calculations

# Define the benchmark
benchmark_ticker = 'SPY'

# ** Data Processing Loop for Each Ticker **
for ticker in tickers:
    # ** 1. Data Downloading Section **
    try:
        data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

        # Check if the data is empty
        if data.empty:
            print(f"No data found for {ticker}. Skipping.")
            continue
        
        # Add benchmark data for Beta calculation
        if ticker == benchmark_ticker:
            benchmark_data = data['Close']
            continue  # Skip the rest for the benchmark itself

        # ** 2. Moving Averages Section **
        data['3_SMA'] = data['Close'].rolling(window=3).mean()
        data['5_SMA'] = data['Close'].rolling(window=5).mean()
        data['20_SMA'] = data['Close'].rolling(window=20).mean()
        data['50_SMA'] = data['Close'].rolling(window=50).mean()

        # ** 3. Market Trend Section **
        data['Market_Trend'] = 0
        data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
        data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

        # ** 4. Price Change Calculations Section **
        data['Price_Change%'] = data['Close'].pct_change() * 100 
        data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

        # ** 5. Rolling Maximum Drawdown Calculation Section **
        data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
        data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

        # ** 6. Cumulative Change Calculation Section using vectorized operations **
        current_trend_shifted = data['Market_Trend'].shift(1)
        current_base_price = data['Close'].shift(1).where(current_trend_shifted != data['Market_Trend']).ffill()
        data['Cumulative_Change%'] = ((data['Close'] - current_base_price) / current_base_price) * 100

        # ** 7. Rolling Up/Down Ratio Calculation **
        data['Up_Day'] = data['Price_Change%'].apply(lambda x: x if x > 0 else 0)
        data['Down_Day'] = data['Price_Change%'].apply(lambda x: -x if x < 0 else 0)
        data['Rolling_Up_Down_Ratio'] = data['Up_Day'].rolling(window=rolling_window).mean() / data['Down_Day'].rolling(window=rolling_window).mean()
        
        # ** 8. Rolling True Strength Index (TSI) Calculation **
        data['Smooth_Price_Change'] = data['Price_Change%'].ewm(span=25, adjust=False).mean()  # Fast TSI
        data['Smooth_Price_Change_Slow'] = data['Price_Change%'].ewm(span=13, adjust=False).mean()  # Slow TSI
        data['TSI'] = 100 * (data['Smooth_Price_Change'] - data['Smooth_Price_Change_Slow']) / data['Smooth_Price_Change_Slow']
        data['Rolling_TSI'] = data['TSI'].rolling(window=rolling_window).mean()

        # ** 9. Rolling Volatility Calculation **
        data['Rolling_Volatility'] = data['Price_Change%'].rolling(window=rolling_window).std()

        # ** 10a. Rolling Maximum Consecutive Up Days Calculation **
        data['Consecutive_Up_Days'] = (data['Price_Change%'] > 0).astype(int)
        data['Rolling_Consecutive_Up_Days'] = data['Consecutive_Up_Days'].groupby((data['Consecutive_Up_Days'] != data['Consecutive_Up_Days'].shift()).cumsum()).cumsum()
        
        # ** 10b. Max Rolling Consecutive Up Days in a 20-day Window **
        consecutive_counts = data['Consecutive_Up_Days'].rolling(window=rolling_window).apply(
            lambda x: (x != 0).astype(int).groupby((x != 0).ne((x.shift())).cumsum()).cumsum().max(),
            raw=False
        )
        data['Max_Rolling_Consecutive_Up_Days'] = consecutive_counts

        # ** 11. RSI Calculation (Relative Strength Index) **
        delta = data['Close'].diff(1)
        gain = (delta.where(delta > 0, 0)).rolling(window=rolling_window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=rolling_window).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))

        # ** 12. Momentum Calculation **
        data['Momentum'] = data['Close'].diff(rolling_window)

        # ** 13. ATR Calculation (Average True Range) **
        data['High_Low'] = data['High'] - data['Low']
        data['High_Close'] = (data['High'] - data['Close'].shift(1)).abs()
        data['Low_Close'] = (data['Low'] - data['Close'].shift(1)).abs()
        data['True_Range'] = data[['High_Low', 'High_Close', 'Low_Close']].max(axis=1)
        data['ATR'] = data['True_Range'].rolling(window=rolling_window).mean()

        # ** 14. Beta Calculation **
        if 'benchmark_data' in locals():
            returns = data['Close'].pct_change()
            benchmark_returns = benchmark_data.pct_change()
            covariance = returns.rolling(window=rolling_window).cov(benchmark_returns)
            variance = benchmark_returns.rolling(window=rolling_window).var()

            data['Beta'] = covariance / variance

        # ** 15. Alpha Calculation **
        if 'benchmark_data' in locals():
            average_benchmark_return = benchmark_data.pct_change().rolling(window=rolling_window).mean()
            data['Alpha'] = (data['Price_Change%'].rolling(window=rolling_window).mean() - (data['Beta'] * average_benchmark_return)).fillna(0)

        # ** 16. Preparing the Trend Indicator DataFrame **
        trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%', 
                                'Rolling_Drawdown', 'Rolling_Up_Down_Ratio', 'Rolling_TSI', 
                                'Rolling_Volatility', 'Rolling_Consecutive_Up_Days', 
                                'Max_Rolling_Consecutive_Up_Days', 'RSI', 'Momentum', 'ATR']].dropna()

        # Conditionally adding Alpha and Beta if they exist in the DataFrame
        trend_columns = [
            f'{ticker} Close',
            f'{ticker} daily move%',
            f'{ticker} trend signal',
            f'{ticker} cumulative change since signal',
            f'{ticker} Rolling Max Drawdown',
            f'{ticker} Rolling Up Down Ratio',
            f'{ticker} Rolling TSI',
            f'{ticker} Rolling Volatility',
            f'{ticker} Consecutive Up Days',
            f'{ticker} Max Rolling Consecutive Up Days',
            f'{ticker} RSI',
            f'{ticker} Momentum',
            f'{ticker} ATR'
        ]

        # Check if Alpha and Beta exist in data for column addition
        if 'Alpha' in data.columns:
            trend_indicator[f'{ticker} Alpha'] = data['Alpha']
            trend_columns.append(f'{ticker} Alpha')

        if 'Beta' in data.columns:
            trend_indicator[f'{ticker} Beta'] = data['Beta']
            trend_columns.append(f'{ticker} Beta')

        trend_indicator.columns = trend_columns

        # ** 17. Sortino Ratio Calculation Section **
        average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
        downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
        downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5 if not x.empty else 0)

        # Calculate Sortino Ratio
        data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
        data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

        # ** 18. Calmar Ratio Calculation Section **
        annualized_return = ((1 + average_return / 100) ** 252) - 1
        data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown

        # ** 19. Sharpe Ratio Calculation Section **
        trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
        trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']

        results[ticker] = trend_indicator  # Store the results for this ticker
        
        # ** 20. Treynor Ratio Calculation Section **
        if 'Beta' in data.columns and not data['Beta'].isnull().all():
            average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
            # Treynor Ratio Calculation
            data['Treynor Ratio'] = (average_return - risk_free_rate) / data['Beta']

        # ** 21. Information Ratio Calculation Section **
        if 'Beta' in data.columns and benchmark_returns is not None:
            average_benchmark_return = benchmark_data.pct_change().rolling(window=rolling_window).mean()
            tracking_error = (data['Price_Change%'].rolling(window=rolling_window).mean() - average_benchmark_return).rolling(window=rolling_window).std()
            data['Information Ratio'] = (average_return - average_benchmark_return) / tracking_error

        # # ** 22. Downside and Upside Capture Ratios Calculation Section**
        # downside_returns = data['Price_Change%'][data['Price_Change%'] < 0]
        # upside_returns = data['Price_Change%'][data['Price_Change%'] > 0]

        # # Benchmark Downside and Upside Returns
        # benchmark_downside_returns = benchmark_returns[benchmark_returns < 0]
        # benchmark_upside_returns = benchmark_returns[benchmark_returns > 0]

        # # Calculate Portfolio Downside and Upside Returns
        # portfolio_downside_return = downside_returns.mean()
        # portfolio_upside_return = upside_returns.mean()

        # benchmark_downside_return = benchmark_downside_returns.mean() if len(benchmark_downside_returns) > 0 else 0
        # benchmark_upside_return = benchmark_upside_returns.mean() if len(benchmark_upside_returns) > 0 else 0

        # # Downside Capture Ratio
        # if benchmark_downside_return != 0:
        #     data['Downside Capture Ratio'] = portfolio_downside_return / benchmark_downside_return
        # else:
        #     data['Downside Capture Ratio'] = None

        # # Upside Capture Ratio
        # if benchmark_upside_return != 0:
        #     data['Upside Capture Ratio'] = portfolio_upside_return / benchmark_upside_return
        # else:
        #     data['Upside Capture Ratio'] = None
        
        # ** 22. Downside and Upside Capture Ratios Calculation Section (rolling window) **

        # Calculate rolling downside and upside returns for the portfolio
        data['Portfolio_Downside'] = data['Price_Change%'].rolling(window=rolling_window).apply(
            lambda x: x[x < 0].mean() if len(x[x < 0]) > 0 else 0, raw=False
        )

        data['Portfolio_Upside'] = data['Price_Change%'].rolling(window=rolling_window).apply(
            lambda x: x[x > 0].mean() if len(x[x > 0]) > 0 else 0, raw=False
        )

        # Calculate rolling downside and upside returns for the benchmark
        data['Benchmark_Downside'] = benchmark_returns.rolling(window=rolling_window).apply(
            lambda x: x[x < 0].mean() if len(x[x < 0]) > 0 else 0, raw=False
        )

        data['Benchmark_Upside'] = benchmark_returns.rolling(window=rolling_window).apply(
            lambda x: x[x > 0].mean() if len(x[x > 0]) > 0 else 0, raw=False
        )

        # Calculate Rolling Downside Capture Ratio
        data['Rolling_Downside_Capture_Ratio'] = data['Portfolio_Downside'] / data['Benchmark_Downside']
        data['Rolling_Downside_Capture_Ratio'] = data['Rolling_Downside_Capture_Ratio'].replace([float('inf'), -float('inf')], None)

        # Calculate Rolling Upside Capture Ratio
        data['Rolling_Upside_Capture_Ratio'] = data['Portfolio_Upside'] / data['Benchmark_Upside']
        data['Rolling_Upside_Capture_Ratio'] = data['Rolling_Upside_Capture_Ratio'].replace([float('inf'), -float('inf')], None)

        # Adding additional metrics to the Trend Indicator DataFrame
        trend_indicator[f'{ticker} Rolling_Downside_Capture_Ratio'] = data['Rolling_Downside_Capture_Ratio']
        trend_indicator[f'{ticker} Rolling_Upside_Capture_Ratio'] = data['Rolling_Upside_Capture_Ratio']
        trend_indicator[f'{ticker}Treynor Ratio'] = data['Treynor Ratio']
        trend_indicator[f'{ticker}Information Ratio'] = data['Information Ratio']


    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# ** Risk-Free Rate Calculation Section **
if "^TNX" in results:  # Ensure ^TNX has been processed successfully
    treasury_yield = results["^TNX"]['^TNX Close']
    risk_free_rate = treasury_yield / 100 / 252  # Convert from annual to daily
else:
    print("Warning: ^TNX results not found. Defaulting risk-free rate to 0.")
    risk_free_rate = 0.0  # Fallback risk-free rate

# ** Calculate Sharpe Ratio for all tickers using the risk-free rate **
for ticker in tickers:
    if ticker in results:
        data = results[ticker]
        rolling_average_return = data[f'{ticker} daily move%'].rolling(window=rolling_window).mean()
        rolling_std_dev_returns = data[f'{ticker} daily move%'].rolling(window=rolling_window).std()

        # Define Sharpe Ratio; replace infinities for stability
        data[f'{ticker} Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
        data[f'{ticker} Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities
        results[ticker] = data  # Update results with Sharpe Ratio

# ** 20. Display Results Section **
for ticker in tickers:
    if ticker in results:  # Only display results for tickers that were processed successfully
        print(f"Results for {ticker}:")
        display(results[ticker].tail(10))  # Display the last 10 entries

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Error processing ^VIX: name 'benchmark_returns' is not defined



[*********************100%%**********************]  1 of 1 completed

Error processing ^TNX: name 'benchmark_returns' is not defined



[*********************100%%**********************]  1 of 1 completed

...

,IAU Close,IAU daily move%,IAU trend signal,IAU cumulative change since signal,IAU Rolling Max Drawdown,IAU Rolling Up Down Ratio,IAU Rolling TSI,IAU Rolling Volatility,IAU Consecutive Up Days,IAU Max Rolling Consecutive Up Days,IAU RSI,IAU Momentum,IAU ATR,IAU Alpha,IAU Beta,IAU Sortino Ratio,IAU Calmar Ratio,IAU Sharpe Ratio
Date,,,,,,,,,,,,,,,,,,
2024-10-23,51.290001,-1.213400,1,2.252793,-0.012134,1.434310,1044.160581,0.705793,0,6.0,58.783794,1.040001,0.499499,0.104876,-0.044225,0.166686,1.443685,0.148310
2024-10-24,51.680000,0.760381,1,3.030304,-0.004622,1.488755,1044.662804,0.715866,1,6.0,59.735980,1.180000,0.503499,0.118018,-0.049179,NaN,1.652935,0.164585
2024-10-25,51.799999,0.232196,-1,0.232196,-0.002311,1.847524,1044.262010,0.681743,2,6.0,64.818025,1.709999,0.492499,0.170246,-0.080044,NaN,2.555585,0.249395
2024-10-28,51.779999,-0.038611,1,-0.038611,-0.002696,2.264816,1041.484748,0.646721,0,6.0,69.259249,2.079998,0.473999,0.207217,-0.041000,0.329601,3.270732,0.320109
2024-10-29,52.360001,1.120127,1,1.081084,0.000000,2.293499,1040.568800,0.653293,1,6.0,69.652660,2.150002,0.465499,0.211772,0.101841,NaN,3.366387,0.324081
2024-10-30,52.650002,0.553860,1,1.640931,0.000000,2.456473,1039.251753,0.655939,2,6.0,71.130437,2.430000,0.462499,0.238513,0.074310,NaN,3.931773,0.363475
2024-10-31,51.840000,-1.538464,1,0.077222,-0.015385,1.685484,1041.294929,0.765815,0,6.0,62.538220,1.639999,0.493499,0.163617,0.389836,0.247049,2.435982,0.213478
2024-11-01,51.619999,-0.424385,-1,-0.424385,-0.019563,1.622504,1054.536345,0.772040,0,6.0,61.596358,1.539997,0.491500,0.154450,0.429316,0.237125,2.269749,0.199749
2024-11-04,51.700001,0.154982,-1,-0.270061,-0.018044,1.767604,1055.559739,0.763960,1,6.0,63.567090,1.780003,0.492999,0.178029,0.416996,NaN,2.703137,0.232918


Results for LNTH:


,LNTH Close,LNTH daily move%,LNTH trend signal,LNTH cumulative change since signal,LNTH Rolling Max Drawdown,LNTH Rolling Up Down Ratio,LNTH Rolling TSI,LNTH Rolling Volatility,LNTH Consecutive Up Days,LNTH Max Rolling Consecutive Up Days,LNTH RSI,LNTH Momentum,LNTH ATR,LNTH Alpha,LNTH Beta,LNTH Sortino Ratio,LNTH Calmar Ratio,LNTH Sharpe Ratio
Date,,,,,,,,,,,,,,,,,,
2024-10-23,110.639999,-1.766845,-1,-1.284799,-0.104999,1.175547,-53.647498,2.166352,0,4.0,53.398184,2.690002,3.670600,0.144586,1.337873,0.062210,0.907994,0.067102
2024-10-24,109.809998,-0.750182,-1,-2.025343,-0.111713,1.385302,-55.108722,2.026831,0,4.0,57.291944,5.379997,3.467850,0.270135,1.500429,0.117934,2.006426,0.133656
2024-10-25,110.300003,0.446230,-1,-1.588150,-0.107750,1.217609,-69.519535,1.938268,1,4.0,54.267056,2.940002,3.395850,0.152021,1.584126,NaN,0.964790,0.078897
2024-10-28,110.099998,-0.181328,-1,-1.766598,-0.109367,1.045913,-75.484317,1.876520,0,4.0,50.542465,0.349998,3.428849,0.031785,1.481144,0.014264,0.176257,0.017344
2024-10-29,110.089996,-0.009085,-1,-1.775522,-0.109448,1.393334,-76.818077,1.675660,0,4.0,57.492944,4.250000,3.505599,0.209119,1.101358,0.094995,1.432430,0.125458
2024-10-30,109.379997,-0.644926,-1,-2.408998,-0.115192,1.390559,-76.942294,1.676226,0,4.0,57.391753,4.199997,3.475849,0.208223,1.120721,0.094332,1.423092,0.124779
2024-10-31,109.839996,0.420551,-1,-1.998577,-0.111471,1.307394,-63.678288,1.657104,1,4.0,55.965799,3.279999,3.460849,0.164690,0.651031,NaN,1.054601,0.099320
2024-11-01,107.980003,-1.693366,-1,-3.658100,-0.126517,1.036423,-67.324892,1.689790,0,4.0,50.346666,0.195000,3.472350,0.022680,0.512004,0.011205,0.120231,0.013275
2024-11-04,115.449997,6.917942,1,6.917942,-0.066090,1.748628,-55.964221,2.263268,1,4.0,62.859633,8.849998,3.880149,0.423424,0.247707,NaN,3.898512,0.187032


Results for GEVO:


,GEVO Close,GEVO daily move%,GEVO trend signal,GEVO cumulative change since signal,GEVO Rolling Max Drawdown,GEVO Rolling Up Down Ratio,GEVO Rolling TSI,GEVO Rolling Volatility,GEVO Consecutive Up Days,GEVO Max Rolling Consecutive Up Days,GEVO RSI,GEVO Momentum,GEVO ATR,GEVO Alpha,GEVO Beta,GEVO Sortino Ratio,GEVO Calmar Ratio,GEVO Sharpe Ratio
Date,,,,,,,,,,,,,,,,,,
2024-10-23,2.91,-3.322256,1,78.527613,-0.427165,3.516469,-0.797800,8.067190,0,5.0,75.247525,1.53,0.2795,4.094477,1.325251,0.780555,27399.825174,0.507643
2024-10-24,2.85,-2.061862,-1,-2.061862,-0.438976,3.139489,0.059014,8.170679,0,5.0,73.089697,1.39,0.2865,3.701775,1.120821,0.714974,10563.460106,0.453121
2024-10-25,2.96,3.859654,-1,1.718211,-0.417323,2.756267,1.604783,7.537800,1,5.0,71.777001,1.25,0.2860,3.038248,1.541400,NaN,2096.462287,0.403184
2024-10-28,2.92,-1.351350,-1,0.343642,-0.425197,3.049394,3.234307,7.393853,0,5.0,72.791519,1.29,0.2815,3.204520,1.778431,0.686653,3148.539916,0.433532
2024-10-29,2.70,-7.534247,-1,-7.216496,-0.468504,3.032693,8.493621,7.406901,0,5.0,70.307166,1.19,0.2895,3.196850,0.150841,0.645173,3082.980898,0.431605
2024-10-30,2.55,-5.555559,-1,-12.371138,-0.498031,2.541690,234.551845,7.654088,0,5.0,66.666664,1.02,0.2960,2.852316,0.716069,0.592147,1328.415871,0.372723
2024-10-31,2.33,-8.627452,-1,-19.931276,-0.541339,1.918008,225.545487,8.012678,0,5.0,61.006287,0.70,0.3035,2.094583,2.765616,0.404340,204.938092,0.261421
2024-11-01,2.24,-3.862657,-1,-23.024057,-0.559055,1.706377,218.622851,8.117563,0,5.0,58.695652,0.56,0.3100,1.748722,2.583279,0.333417,86.367332,0.215357
2024-11-04,2.37,5.803566,-1,-18.556707,-0.533465,1.631223,205.451710,7.971319,1,5.0,58.307207,0.53,0.3115,1.561630,3.544958,NaN,54.051586,0.195973


In [43]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze including the benchmark
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GOOG', 'AVGO', 'AMZN', 'NVDA', 'MSFT', 'DIS', 'AAPL', 'WMT', 'META',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM', 'PLTR', 'QUBT', 'IONQ', 'APLD', 'LMND', 'VRT', 'RKLB', 'AFRM', 'FLR', 'ESOA', 'BKNG', 'HQY', 'NU', 'VNO', 'MP', 'SMR', 'IDCC', 'RVMD', 'DHI', 'PANW', 'DASH', 'CHWY', 'WPM',
           'CL', 'LMT', 'TKO', 'T', 'SE', 'GDX', 'COST', 'GOOG', 'SLV', 'NEE', 'CCJ', 'IAU', 'LNTH', 'GEVO', 'FICO']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # Adjust this as needed
rolling_window = 20   # Define the window size for rolling calculations

# Define the benchmark
benchmark_ticker = 'SPY'

# ** Data Processing Loop for Each Ticker **
for ticker in tickers:
    # ** 1. Data Downloading Section **
    try:
        data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

        # Check if the data is empty
        if data.empty:
            print(f"No data found for {ticker}. Skipping.")
            continue
        
        # Add benchmark data for Beta calculation
        if ticker == benchmark_ticker:
            benchmark_data = data['Close']
            continue  # Skip the rest for the benchmark itself

        # ** 2. Moving Averages Section **
        data['3_SMA'] = data['Close'].rolling(window=3).mean()
        data['5_SMA'] = data['Close'].rolling(window=5).mean()
        data['20_SMA'] = data['Close'].rolling(window=20).mean()
        data['50_SMA'] = data['Close'].rolling(window=50).mean()

        # ** 3. Market Trend Section **
        data['Market_Trend'] = 0
        data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
        data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

        # ** 4. Price Change Calculations Section **
        data['Price_Change%'] = data['Close'].pct_change() * 100 
        data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

        # ** 5. Rolling Maximum Drawdown Calculation Section **
        data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
        data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

        # ** 6. Cumulative Change Calculation Section using vectorized operations **
        current_trend_shifted = data['Market_Trend'].shift(1)
        current_base_price = data['Close'].shift(1).where(current_trend_shifted != data['Market_Trend']).ffill()
        data['Cumulative_Change%'] = ((data['Close'] - current_base_price) / current_base_price) * 100

        # ** 7. Rolling Up/Down Ratio Calculation **
        data['Up_Day'] = data['Price_Change%'].apply(lambda x: x if x > 0 else 0)
        data['Down_Day'] = data['Price_Change%'].apply(lambda x: -x if x < 0 else 0)
        data['Rolling_Up_Down_Ratio'] = data['Up_Day'].rolling(window=rolling_window).mean() / data['Down_Day'].rolling(window=rolling_window).mean()
        
        # ** 8. Rolling True Strength Index (TSI) Calculation **
        data['Smooth_Price_Change'] = data['Price_Change%'].ewm(span=25, adjust=False).mean()  # Fast TSI
        data['Smooth_Price_Change_Slow'] = data['Price_Change%'].ewm(span=13, adjust=False).mean()  # Slow TSI
        data['TSI'] = 100 * (data['Smooth_Price_Change'] - data['Smooth_Price_Change_Slow']) / data['Smooth_Price_Change_Slow']
        data['Rolling_TSI'] = data['TSI'].rolling(window=rolling_window).mean()

        # ** 9. Rolling Volatility Calculation **
        data['Rolling_Volatility'] = data['Price_Change%'].rolling(window=rolling_window).std()

        # ** 10a. Rolling Maximum Consecutive Up Days Calculation **
        data['Consecutive_Up_Days'] = (data['Price_Change%'] > 0).astype(int)
        data['Rolling_Consecutive_Up_Days'] = data['Consecutive_Up_Days'].groupby((data['Consecutive_Up_Days'] != data['Consecutive_Up_Days'].shift()).cumsum()).cumsum()
        
        # ** 10b. Max Rolling Consecutive Up Days in a 20-day Window **
        consecutive_counts = data['Consecutive_Up_Days'].rolling(window=rolling_window).apply(
            lambda x: (x != 0).astype(int).groupby((x != 0).ne((x.shift())).cumsum()).cumsum().max(),
            raw=False
        )
        data['Max_Rolling_Consecutive_Up_Days'] = consecutive_counts

        # ** 11. RSI Calculation (Relative Strength Index) **
        delta = data['Close'].diff(1)
        gain = (delta.where(delta > 0, 0)).rolling(window=rolling_window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=rolling_window).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))

        # ** 12. Momentum Calculation **
        data['Momentum'] = data['Close'].diff(rolling_window)

        # ** 13. ATR Calculation (Average True Range) **
        data['High_Low'] = data['High'] - data['Low']
        data['High_Close'] = (data['High'] - data['Close'].shift(1)).abs()
        data['Low_Close'] = (data['Low'] - data['Close'].shift(1)).abs()
        data['True_Range'] = data[['High_Low', 'High_Close', 'Low_Close']].max(axis=1)
        data['ATR'] = data['True_Range'].rolling(window=rolling_window).mean()

        # ** 14. Beta Calculation **
        if 'benchmark_data' in locals():
            returns = data['Close'].pct_change()
            benchmark_returns = benchmark_data.pct_change()
            covariance = returns.rolling(window=rolling_window).cov(benchmark_returns)
            variance = benchmark_returns.rolling(window=rolling_window).var()

            data['Beta'] = covariance / variance

        # ** 15. Alpha Calculation **
        if 'benchmark_data' in locals():
            average_benchmark_return = benchmark_data.pct_change().rolling(window=rolling_window).mean()
            data['Alpha'] = (data['Price_Change%'].rolling(window=rolling_window).mean() - (data['Beta'] * average_benchmark_return)).fillna(0)

        # ** 16. Preparing the Trend Indicator DataFrame **
        trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%', 
                                'Rolling_Drawdown', 'Rolling_Up_Down_Ratio', 'Rolling_TSI', 
                                'Rolling_Volatility', 'Rolling_Consecutive_Up_Days', 
                                'Max_Rolling_Consecutive_Up_Days', 'RSI', 'Momentum', 'ATR']].dropna()

        # Conditionally adding Alpha and Beta if they exist in the DataFrame
        trend_columns = [
            f'{ticker} Close',
            f'{ticker} daily move%',
            f'{ticker} trend signal',
            f'{ticker} cumulative change since signal',
            f'{ticker} Rolling Max Drawdown',
            f'{ticker} Rolling Up Down Ratio',
            f'{ticker} Rolling TSI',
            f'{ticker} Rolling Volatility',
            f'{ticker} Consecutive Up Days',
            f'{ticker} Max Rolling Consecutive Up Days',
            f'{ticker} RSI',
            f'{ticker} Momentum',
            f'{ticker} ATR'
        ]

        # Check if Alpha and Beta exist in data for column addition
        if 'Alpha' in data.columns:
            trend_indicator[f'{ticker} Alpha'] = data['Alpha']
            trend_columns.append(f'{ticker} Alpha')

        if 'Beta' in data.columns:
            trend_indicator[f'{ticker} Beta'] = data['Beta']
            trend_columns.append(f'{ticker} Beta')

        trend_indicator.columns = trend_columns

        # ** 17. Sortino Ratio Calculation Section **
        average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
        downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
        downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5 if not x.empty else 0)

        # Calculate Sortino Ratio
        data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
        data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

        # ** 18. Calmar Ratio Calculation Section **
        annualized_return = ((1 + average_return / 100) ** 252) - 1
        data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown

        # ** 19. Sharpe Ratio Calculation Section **
        trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
        trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']

        results[ticker] = trend_indicator  # Store the results for this ticker

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# ** Risk-Free Rate Calculation Section **
if "^TNX" in results:  # Ensure ^TNX has been processed successfully
    treasury_yield = results["^TNX"]['^TNX Close']
    risk_free_rate = treasury_yield / 100 / 252  # Convert from annual to daily
else:
    print("Warning: ^TNX results not found. Defaulting risk-free rate to 0.")
    risk_free_rate = 0.0  # Fallback risk-free rate

# ** Calculate Sharpe Ratio for all tickers using the risk-free rate **
for ticker in tickers:
    if ticker in results:
        data = results[ticker]
        rolling_average_return = data[f'{ticker} daily move%'].rolling(window=rolling_window).mean()
        rolling_std_dev_returns = data[f'{ticker} daily move%'].rolling(window=rolling_window).std()

        # Define Sharpe Ratio; replace infinities for stability
        data[f'{ticker} Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
        data[f'{ticker} Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities
        results[ticker] = data  # Update results with Sharpe Ratio

# ** 20. Display Results Section **
# for ticker in tickers:
#     if ticker in results:  # Only display results for tickers that were processed successfully
#         print(f"Results for {ticker}:")
#         display(results[ticker].tail(2))  # Display the last 10 entries

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

No data found for AFRM. Skipping.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******